In [1]:
!pip install --upgrade pyttsx3
!pip install gtts
!pip install pydub
!apt-get install ffmpeg
!pip install googletrans
!pip install -U -q google-generativeai==0.7.2

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


In [39]:
import google.generativeai as genai
import requests
import json
from gtts import gTTS
from IPython.display import Audio, display
import re

# Configuración de API Keys (Reemplazar con valores reales)
GOOGLE_API_KEY = 
API_KEY = 
API_SECRET = 

# Configurar Google Gemini
genai.configure(api_key=GOOGLE_API_KEY)
version = 'gemini-2.0-flash-exp'
model = genai.GenerativeModel(version)

# Función para traducir nombres de ciudades a inglés
def traducir_ciudad(ciudad):
    url = "https://translate.googleapis.com/translate_a/single"
    params = {
        "client": "gtx",
        "sl": "es",
        "tl": "en",
        "dt": "t",
        "q": ciudad
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        try:
            return response.json()[0][0][0]  # Extraer la traducción
        except:
            return ciudad  # En caso de error, devolver la ciudad original
    return ciudad

# Función para obtener el código IATA de la ciudad
def obtener_codigo_iata(ciudad):
    if not ciudad:
        return None
    ciudad_traducida = traducir_ciudad(ciudad)
    url = "https://www.air-port-codes.com/api/v1/multi"
    headers = {
        "APC-Auth": API_KEY,
        "APC-Auth-Secret": API_SECRET
    }
    params = {"term": ciudad_traducida}

    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        if "airports" in data and data["airports"]:
            return data["airports"][0]["iata"], data["airports"][0]["name"]  # Retornar código IATA y nombre del aeropuerto
    return None, None

# Función para hablar el texto utilizando gTTS
def responder_con_voz(texto):
    tts = gTTS(text=texto, lang='es')
    tts.save("respuesta.mp3")  # Guardamos el archivo de audio
    return "respuesta.mp3"

# Función para reproducir audio de manera asíncrona
def reproducir_audio(archivo_audio):
    display(Audio(archivo_audio, autoplay=True))

# Función para inferir la mejor aerolínea
def inferir_mejor_aerolinea(origen, destino, fecha):
    # Aquí podrías integrar una API de búsqueda de vuelos para obtener datos reales
    # Por ahora, simulamos una inferencia basada en datos estáticos
    aerolineas = {
        "Delta Airlines": {"precio": 300, "puntuacion": 4.5},
        "American Airlines": {"precio": 320, "puntuacion": 4.3},
        "United Airlines": {"precio": 310, "puntuacion": 4.4},
    }
    mejor_aerolinea = max(aerolineas.keys(), key=lambda x: (aerolineas[x]["puntuacion"], -aerolineas[x]["precio"]))
    return mejor_aerolinea

# Función principal del asistente
def asistente():
    print("Hola, bienvenido a Sky2travel. ¿Cómo te puedo ayudar?")
    archivo_audio = responder_con_voz("Hola, bienvenido a Sky2travel. ¿Cómo te puedo ayudar?")
    reproducir_audio(archivo_audio)  # Reproducir el audio de bienvenida

    while True:
        mensaje = input("Tú: ")
        if mensaje.lower() == "salir":
            print("Gracias por usar Sky2travel. ¡Hasta luego!")
            archivo_audio = responder_con_voz("Gracias por usar Sky2travel. ¡Hasta luego!")
            reproducir_audio(archivo_audio)  # Reproducir el audio de despedida
            break

        # Verificar si el mensaje es inapropiado
        temas_prohibidos = ["arma", "violencia", "drogas", "ilegal", "delito", "asesinato", "bomba", "terrorismo"]
        if any(tema in mensaje.lower() for tema in temas_prohibidos):
            respuesta = "Lo siento, pero solo puedo ayudarte con temas relacionados con viajes y vuelos."
            print(respuesta)
            archivo_audio = responder_con_voz(respuesta)
            reproducir_audio(archivo_audio)  # Reproducir el audio de advertencia
            continue

        # Enviar todo el mensaje al modelo sin procesamiento previo
        prompt = f"""
        Eres un asistente de viajes para la empresa Sky2travel. Solo puedes responder preguntas relacionadas con viajes y vuelos.
        No respondas preguntas sobre armas, violencia, temas ilegales u ofensivos.
        Si el usuario pregunta sobre estos temas, responde con: 'Lo siento, pero solo puedo ayudarte con temas relacionados con viajes y vuelos.'

        Un usuario ha realizado la siguiente consulta:
        "{mensaje}"

        Extrae los siguientes datos y genera una respuesta estrictamente en formato JSON con estos campos:
        {{
            "origen": "Nombre de la ciudad o país de origen (si no se menciona explícitamente, intenta inferirlo de contexto o responde 'Desconocido')",
            "destino": "Nombre de la ciudad o país de destino",
            "fecha": "dd-mm-yyyy" o null si no se menciona,
            "cantidad": Número de pasajeros (por defecto 1),
            "aerolinea": "Nombre de la aerolínea" o null si no se menciona
        }}

        Si el usuario menciona solo el destino y no el origen, intenta inferir el origen con base en un contexto general o devuelve 'Desconocido'.
        Asegúrate de que los nombres de ciudades o países estén en inglés.

        Solo devuelve el JSON sin texto adicional. No agregues comentarios ni explicaciones.
        """

        response = model.generate_content(prompt, generation_config={"temperature": 0.2, "top_p": 0.9})

        if not response or not response.text:
            respuesta = "Error: No se recibió respuesta válida del modelo."
            print(respuesta)
            archivo_audio = responder_con_voz(respuesta)
            reproducir_audio(archivo_audio)  # Reproducir el audio de error
            continue

        try:
            # Limpiar la respuesta para asegurar un JSON válido
            response_text = response.text.strip("```json").strip("```").strip()
            response_json = json.loads(response_text)  # Convertir la respuesta en JSON
        except json.JSONDecodeError:
            # Si el JSON no es válido, intentar extraer manualmente los campos
            print("Advertencia: La respuesta del modelo no es un JSON válido. Intentando extraer datos manualmente...")
            try:
                # Extraer datos manualmente usando expresiones regulares
                origen = re.search(r'"origen":\s*"([^"]+)"', response.text)
                destino = re.search(r'"destino":\s*"([^"]+)"', response.text)
                fecha = re.search(r'"fecha":\s*"([^"]+)"', response.text)
                cantidad = re.search(r'"cantidad":\s*(\d+)', response.text)
                aerolinea = re.search(r'"aerolinea":\s*"([^"]+)"', response.text)

                response_json = {
                    "origen": origen.group(1) if origen else "Desconocido",
                    "destino": destino.group(1) if destino else None,
                    "fecha": fecha.group(1) if fecha else None,
                    "cantidad": int(cantidad.group(1)) if cantidad else 1,
                    "aerolinea": aerolinea.group(1) if aerolinea else None
                }
            except Exception as e:
                print(f"Error al extraer datos manualmente: {e}")
                respuesta = "Lo siento, no pude procesar tu solicitud. Por favor, intenta de nuevo."
                print(respuesta)
                archivo_audio = responder_con_voz(respuesta)
                reproducir_audio(archivo_audio)  # Reproducir el audio de error
                continue

        # Verificar si el origen es "Desconocido" y pedir al usuario que lo ingrese
        if response_json.get("origen") == "Desconocido":
            origen = input("Por favor, ingresa la ciudad o país de origen: ")
            response_json["origen"] = origen

        # Obtener códigos IATA y nombres de aeropuertos
        origen_nombre = traducir_ciudad(response_json.get("origen", ""))
        destino_nombre = traducir_ciudad(response_json.get("destino", ""))
        origen_iata, nombre_aeropuerto_origen = obtener_codigo_iata(origen_nombre)
        destino_iata, nombre_aeropuerto_destino = obtener_codigo_iata(destino_nombre)

        fecha = response_json.get('fecha') or 'una fecha a definir'
        aerolinea = response_json.get('aerolinea') or inferir_mejor_aerolinea(origen_nombre, destino_nombre, fecha)

        # Mensaje de confirmación
        respuesta = f"Perfecto, comienzo la búsqueda de tu viaje a {destino_nombre} desde {origen_nombre} para el {fecha} con {aerolinea}."
        if nombre_aeropuerto_origen:
            respuesta += f" El aeropuerto de origen es {nombre_aeropuerto_origen} ({origen_iata}) y el de destino es {nombre_aeropuerto_destino} ({destino_iata})."
        print(respuesta)
        archivo_audio = responder_con_voz(respuesta)
        reproducir_audio(archivo_audio)  # Reproducir el audio de confirmación

        # Imprimir la información de manera estructurada
        print("\n--- Información del Viaje ---")
        print(f"Origen: {origen_nombre}")
        if nombre_aeropuerto_origen:
            print(f"Aeropuerto de Origen: {nombre_aeropuerto_origen} ({origen_iata})")
        print(f"Destino: {destino_nombre}")
        if nombre_aeropuerto_destino:
            print(f"Aeropuerto de Destino: {nombre_aeropuerto_destino} ({destino_iata})")
        print(f"Fecha: {fecha}")
        print(f"Pax: {response_json.get('cantidad', 1)}")
        print(f"Aerolínea: {aerolinea}")
        print("-----------------------------\n")

        # Preguntar si desea buscar otro viaje o salir
        while True:
            opcion = input("¿Deseas buscar otro viaje? (si/no): ").lower()
            if opcion in ["si", "no"]:
                break
            else:
                print("Por favor, responde con 'si' o 'no'.")

        if opcion == "no":
            print("Gracias por usar Sky2travel. ¡Hasta luego!")
            archivo_audio = responder_con_voz("Gracias por usar Sky2travel. ¡Hasta luego!")
            reproducir_audio(archivo_audio)  # Reproducir el audio de despedida
            break

# Ejecutar el asistente
asistente()

Hola, bienvenido a Sky2travel. ¿Cómo te puedo ayudar?


Tú: Billete barato AirEuropa de Madrid a Sevilla
Perfecto, comienzo la búsqueda de tu viaje a Seville desde Madrid para el una fecha a definir con AirEuropa. El aeropuerto de origen es Adolfo Suárez Madrid–Barajas Airport (MAD) y el de destino es Seville Airport (SVQ).



--- Información del Viaje ---
Origen: Madrid
Aeropuerto de Origen: Adolfo Suárez Madrid–Barajas Airport (MAD)
Destino: Seville
Aeropuerto de Destino: Seville Airport (SVQ)
Fecha: una fecha a definir
Pax: 1
Aerolínea: AirEuropa
-----------------------------

¿Deseas buscar otro viaje? (si/no): no
Gracias por usar Sky2travel. ¡Hasta luego!


# Sky2Travel - Asistente de Viajes

Sky2Travel es un asistente virtual diseñado para ayudar a los usuarios a buscar vuelos y planificar viajes. Utilizando una combinación de APIs y modelos de lenguaje avanzados, el proyecto ofrece una experiencia interactiva y en tiempo real que incluye procesamiento de lenguaje natural, traducción automática, generación de respuestas estructuradas y síntesis de voz.

## Funcionalidades Clave

- **Interfaz Conversacional:**  
  El asistente interactúa con el usuario a través de mensajes de texto y voz. Al iniciar, saluda al usuario y, conforme se desarrollan las consultas, utiliza respuestas generadas por voz para hacer la experiencia más dinámica.

- **Procesamiento del Lenguaje Natural:**  
  Se utiliza el modelo generativo de Google Gemini para interpretar las consultas del usuario. El mensaje se envía junto con un *prompt* cuidadosamente diseñado que:
  - Define el rol del asistente como experto en viajes.
  - Especifica que solo se deben tratar temas relacionados con vuelos y viajes.
  - Indica al modelo que extraiga información esencial (origen, destino, fecha, cantidad de pasajeros y aerolínea) en formato JSON.

- **Manejo de Datos y Validación:**  
  Una vez recibido el JSON del modelo, el código:
  - Valida y procesa la respuesta.
  - Si la respuesta no es un JSON válido, se utiliza una extracción manual con expresiones regulares para obtener los campos necesarios.
  - Si el origen es desconocido, se solicita al usuario que lo ingrese manualmente.

- **Integración de APIs Externas:**
  - **Traducción de Ciudades:**  
    Se usa la API de Google Translate para convertir nombres de ciudades al inglés, asegurando una búsqueda adecuada en las APIs de aeropuertos.
  - **Obtención de Códigos IATA:**  
    La API de Air-Port-Codes se utiliza para obtener el código IATA y el nombre del aeropuerto, tanto de origen como de destino.
  
- **Síntesis de Voz:**  
  La biblioteca gTTS (Google Text-to-Speech) se integra para convertir respuestas de texto a archivos de audio, proporcionando mensajes de bienvenida, confirmación y error a través de voz.

- **Filtrado de Contenidos Inapropiados:**  
  Se implementa un sistema de verificación para evitar respuestas sobre temas no relacionados (como armas o violencia). Si se detecta alguno de estos términos, el asistente responde indicando que solo puede ayudar en temas de viajes.

## Detalle del *Prompt* al Modelo

El *prompt* enviado al modelo de Google Gemini es esencial para estructurar la respuesta. Este *prompt*:

- Define al asistente como un experto en viajes, especificando claramente que solo debe tratar temas relacionados con vuelos.
- Instruye al modelo para que extraiga información crítica en un formato JSON específico:
  - **origen:** Ciudad o país de origen. Si no se menciona, se intenta inferir o se devuelve "Desconocido".
  - **destino:** Ciudad o país de destino.
  - **fecha:** Fecha del viaje en formato `dd-mm-yyyy` o `null` si no se menciona.
  - **cantidad:** Número de pasajeros (por defecto 1).
  - **aerolinea:** Nombre de la aerolínea o `null` si no se menciona.
  
Esta estructura permite que, independientemente de cómo se formule la pregunta del usuario, se extraigan y organicen los datos necesarios para continuar el proceso de búsqueda y confirmación del viaje.

## Flujo de Trabajo

### Inicio y Bienvenida:
Al iniciar, el asistente saluda tanto por texto como por voz, invitando al usuario a hacer su consulta.

### Consulta del Usuario y Procesamiento:
- El usuario ingresa una consulta de viaje.
- Se verifica si el mensaje contiene temas no permitidos. Si es así, se notifica al usuario.
- La consulta se envía al modelo de Google Gemini junto con el prompt definido.
- Se procesa la respuesta JSON para extraer los datos relevantes.

### Validación y Obtención de Datos Externos:
- Si el origen no está definido, se solicita al usuario que lo ingrese.
- Se traducen las ciudades al inglés para realizar búsquedas en la API de aeropuertos.
- Se obtienen los códigos IATA y nombres de los aeropuertos.

### Confirmación y Respuesta:
El asistente confirma la búsqueda del vuelo, proporcionando detalles como:
- Aeropuerto de origen y destino (con sus códigos IATA).
- Fecha del viaje.
- Número de pasajeros.
- Aerolínea (inferida en caso de no ser especificada).

### Ciclo de Consulta:
Tras completar una búsqueda, se pregunta al usuario si desea realizar otra consulta o salir.



